<a href="https://colab.research.google.com/github/Rishiatweb/Chronos-data-Scribe/blob/main/Chronos_Data_Scribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#!pip uninstall openai -y
#!pip install openai==0.28
#!pip install pandas python-dotenv Jinja2
#import openai
#print(f"OpenAI version after explicit install: {openai.__version__}")
!pip install pandas python-dotenv Jinja2 transformers torch sentencepiece

In [22]:
import os
from google.colab import userdata # For Colab secrets

#from openai import OpenAI
#client = OpenAI(api_key=OPENAI_API_KEY)

# Load secrets
#OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
EMAIL_HOST = userdata.get('EMAIL_HOST')
EMAIL_PORT = int(userdata.get('EMAIL_PORT')) # Ensure port is an integer
EMAIL_USER = userdata.get('EMAIL_USER')
EMAIL_PASSWORD = userdata.get('EMAIL_PASSWORD')
EMAIL_RECIPIENT = userdata.get('EMAIL_RECIPIENT')

# OpenAI API configuration
import openai
openai.api_key = OPENAI_API_KEY

# Other Configurations
#LLM_MODEL = "gpt-3.5-turbo-instruct" # Or "gpt-3.5-turbo", "gpt-4"
#SAMPLE_ROWS = 5

#Jinja2 email template
EMAIL_TEMPLATE_STR = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>New Data Report</title>
    <style>
        body { font-family: sans-serif; margin: 20px; color: #333; }
        .container { background-color: #f9f9f9; border: 1px solid #ddd; padding: 20px; border-radius: 5px; }
        h2 { color: #2c3e50; }
        ul { list-style-type: none; padding-left: 0; }
        li strong { color: #3498db; }
        .summary { background-color: #e8f4f8; border-left: 4px solid #3498db; padding: 10px; margin-top: 15px; font-style: italic;}
        .footer { margin-top: 20px; font-size: 0.8em; color: #7f8c8d; }
    </style>
</head>
<body>
    <div class="container">
        <h2>Chronos Data Scribe Report</h2>
        <p>Greetings!</p>
        <p>A new data file has been processed for you:</p>
        <ul>
            <li><strong>File Name:</strong> {{ filename }}</li>
            <li><strong>Detected At:</strong> {{ detected_at }}</li>
            <li><strong>Full Path:</strong> {{ file_path }}</li>
        </ul>
        <h3>AI-Generated Summary:</h3>
        <div class="summary">
            <p>{{ summary }}</p>
        </div>
        <p class="footer">This is an automated notification from Chronos Data Scribe.</p>
    </div>
</body>
</html>
"""

# Logging setup
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger("ChronosScribeColab")

print("Configuration loaded.")

Configuration loaded.


In [23]:
csv_content = """ID,Name,Value,Date
ID,Name,Value,Date
1,Project Alpha,150,2023-01-10
2,Project Beta,220,2023-01-15
3,Project Alpha,180,2023-01-20
4,Project Gamma,300,2023-01-25
5,Project Beta,250,2023-02-01
6,Project Alpha,160,2023-02-05
7,Project Delta,450,2023-02-10
8,Project Beta,230,2023-02-15
9,Project Alpha,190,2023-02-20
10,Project Gamma,320,2023-02-25
11,Project Epsilon,500,2023-03-01
12,Project Alpha,170,2023-03-05
13,Project Beta,240,2023-03-10
14,Project Delta,470,2023-03-15
15,Project Alpha,200,2023-03-20
16,Project Gamma,310,2023-03-22
17,Project Beta,210,2023-03-25
18,Project Epsilon,520,2023-03-28
19,Project Alpha,175,2023-04-02
20,Project Delta,460,2023-04-05
21,Data Task A,90,2023-01-12
22,Data Task B,110,2023-01-18
23,Data Task A,95,2023-02-03
24,Data Task C,130,2023-02-12
25,Data Task B,105,2023-03-07
"""
csv_filename = "sample_data.csv"
with open(csv_filename, "w") as f:
    f.write(csv_content)

print(f"Created dummy CSV: {csv_filename}")

Created dummy CSV: sample_data.csv


In [24]:
import pandas as pd

def get_csv_sample_and_headers(file_path):
    """Reads CSV, extracts headers and a sample of rows."""
    try:
        df = pd.read_csv(file_path, nrows=SAMPLE_ROWS * 2 if SAMPLE_ROWS > 0 else None) # Read a bit more or all for small samples
        if df.empty:
             logger.warning(f"CSV file {file_path} is empty or only headers.")
             return "No headers (empty/header-only file)", "No data (empty/header-only file)", "No data (empty/header-only file)", []

        headers_str = ", ".join(df.columns.tolist())
        column_list = df.columns.tolist()

        if len(df) > SAMPLE_ROWS:
            sample_head_df = df.head(SAMPLE_ROWS)
            sample_tail_df = df.tail(SAMPLE_ROWS) if len(df) > SAMPLE_ROWS else pd.DataFrame()
            sample_data_head_str = sample_head_df.to_string(index=False)
            sample_data_tail_str = sample_tail_df.to_string(index=False) if not sample_tail_df.empty else "Not enough data for tail sample."
        else:
            sample_data_head_str = df.to_string(index=False)
            sample_data_tail_str = "Not enough data for tail sample."

        return headers_str, sample_data_head_str, sample_data_tail_str, column_list
    except pd.errors.EmptyDataError:
        logger.warning(f"CSV file {file_path} is completely empty.")
        return "No headers (empty file)", "No data (empty file)", "No data (empty file)", []
    except Exception as e:
        logger.error(f"Error reading CSV {file_path}: {e}")
        raise

In [25]:
from transformers import pipeline
import pandas as pd
import os

def generate_summary_with_huggingface(filepath, headers_str, sample_data_head_str, sample_data_tail_str, column_list): # Added filepath
    """Generates summary using a local Hugging Face model."""
    if summarizer_pipeline is None:
        logger.error("Hugging Face summarizer pipeline not loaded. Cannot generate summary.")
        return "Could not generate summary: Summarization model failed to load."

    filename = os.path.basename(filepath)

    intro_sentence = f"The following data is from a file named '{filename}'. "
    description_sentence = f"It contains records with fields: {headers_str}. "
    data_representation = "For example, some records show: \n"

    try:
        df_sample = pd.read_csv(filepath, nrows=SAMPLE_ROWS)
        formatted_samples = []
        if not df_sample.empty:
            for i, row in df_sample.iterrows():
                row_desc_parts = []
                for col_name in df_sample.columns: # Iterate in original column order
                    row_desc_parts.append(f"{col_name} is '{row[col_name]}'")
                formatted_samples.append("One record shows " + ", and ".join(row_desc_parts) + ".")
            data_representation += "\n".join(formatted_samples)
        else:
            data_representation += "The file sample is empty."
    except Exception as e:
        logger.error(f"Error re-reading CSV for formatting: {e}")
        data_representation += "Could not display sample records due to an error."


    text_to_summarize = intro_sentence + description_sentence + data_representation

    MAX_INPUT_CHARS = 3500
    if len(text_to_summarize) > MAX_INPUT_CHARS:
        logger.warning(f"Input text for {filename} is too long ({len(text_to_summarize)} chars), truncating to {MAX_INPUT_CHARS} chars.")
        text_to_summarize = text_to_summarize[:MAX_INPUT_CHARS]

    logger.info(f"Text to summarize for {filename}:\n{text_to_summarize}") # Log the input

    try:
        logger.info(f"Generating summary for {filename} using Hugging Face model.")
        summary_output = summarizer_pipeline(text_to_summarize, max_length=100, min_length=25, do_sample=False)

        if summary_output and isinstance(summary_output, list) and 'summary_text' in summary_output[0]:
            summary = summary_output[0]['summary_text']
            logger.info(f"Summary received for {filename}")
            return summary.strip()
        else:
            logger.error(f"Hugging Face summarizer returned unexpected output format: {summary_output}")
            return "Could not generate summary (HF model returned unexpected format)."
    except Exception as e:
        logger.error(f"Error calling Hugging Face summarizer for {filename}: {e}")
        return f"Could not generate AI summary due to an error with the local model: {e}"

In [26]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from jinja2 import Environment, BaseLoader
from datetime import datetime

def send_notification_email(filename, file_path_in_colab, summary_text):
    """Sends an email notification."""
    if not all([EMAIL_HOST, EMAIL_PORT, EMAIL_USER, EMAIL_PASSWORD, EMAIL_RECIPIENT]):
        logger.error("Email configuration is incomplete. Cannot send notification.")
        return

    # Use Jinja2 with the template string defined earlier
    jinja_env = Environment(loader=BaseLoader()) # BaseLoader for string templates
    template = jinja_env.from_string(EMAIL_TEMPLATE_STR)

    html_content = template.render(
        filename=filename,
        detected_at=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        file_path=file_path_in_colab,
        summary=summary_text
    )

    msg = MIMEMultipart('alternative')
    msg['Subject'] = f"[Chronos Scribe Colab] New Data Report: {filename}"
    msg['From'] = EMAIL_USER
    msg['To'] = EMAIL_RECIPIENT
    msg.attach(MIMEText(html_content, 'html'))

    try:
        with smtplib.SMTP(EMAIL_HOST, EMAIL_PORT) as server:
            server.starttls()
            server.login(EMAIL_USER, EMAIL_PASSWORD)
            server.sendmail(EMAIL_USER, EMAIL_RECIPIENT, msg.as_string())
        logger.info(f"Notification email sent for {filename} to {EMAIL_RECIPIENT}")
    except Exception as e:
        logger.error(f"Failed to send email: {e}")

In [27]:
def process_single_file(filepath):
    filename = os.path.basename(filepath)
    logger.info(f"--- Processing {filename} ---")

    try:
        headers, sample_head, sample_tail, col_list = get_csv_sample_and_headers(filepath)

        if headers == "No headers (empty file)" or headers == "No headers (empty/header-only file)":
            logger.info(f"Skipping {filename} as it's empty or has no data rows.")
            return

        summary = generate_summary_with_huggingface(filename, headers, sample_head, sample_tail, col_list)
        print(f"\nGenerated Summary for {filename}:\n{summary}\n")

        send_notification_email(filename, filepath, summary)

    except Exception as e:
        logger.error(f"Error processing file {filename}: {e}")
        send_notification_email(filename, filepath, f"Error processing file: {e}. Manual review needed.")

# --- Test with the dummy CSV ---
test_csv_path = csv_filename
process_single_file(test_csv_path)


Generated Summary for sample_data.csv:
The following data is from a file named 'sample_data.csv' It contains records with fields: ID, Name, Value, Date . Some records show ID is 'ID', and Name is 'Name', and Value is 'Value', and Date is 'Date'

